# Variational encoder model

## import lib


In [114]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader , TensorDataset
from torchvision.utils import save_image, make_grid
from torch.optim import Adam
import cudnn

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
import matplotlib.cm as cm

import copy
import seaborn as sns

from scipy.stats import norm
from sklearn.neighbors import KernelDensity, LocalOutlierFactor

import tqdm


## ML pipeline

### Step 0. Model Hyperparameters

In [597]:
# model hyperparameters
cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if cuda else "cpu")

batch_size = 100
x_dim = 19
hidden_dim = 1024
latent_dim = 19

lr = 0.5e-3
epochs = 30

training_seed = 21

state_dim = 64
action_dim = 19

### Step 1. Load Dataset

get data from file

In [598]:
num_seeds = 30
all_state_dim = 64
seed = 0
# Load fullstate
data_fullstate = np.empty(num_seeds, dtype=object)
data_no_joint_pos = np.empty(num_seeds, dtype=object)
data_no_joint_vel = np.empty(num_seeds, dtype=object)
data_no_action = np.empty(num_seeds, dtype=object)
data_no_imu = np.empty(num_seeds, dtype=object)
data_no_fc = np.empty(num_seeds, dtype=object)
for i in range(num_seeds):
    data_fullstate[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-fullstate-rand-{i}.npz")    
    data_no_joint_pos[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-no_joint_pos-rand-{i}.npz")
    data_no_joint_vel[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-no_joint_vel-rand-{i}.npz")
    data_no_action[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-no_action-rand-{i}.npz")
    data_no_imu[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-no_imu-rand-{i}.npz")
    data_no_fc[i] = np.load(f"data/HEBB-FULL-STATE_seed-{seed}-no_fc-rand-{i}.npz")

In [599]:
train_x = torch.empty((0, all_state_dim), dtype=torch.float32 ,device=DEVICE)
train_y = torch.empty((0, action_dim), dtype=torch.float32,device=DEVICE)
test_x = torch.empty((0, all_state_dim), dtype=torch.float32,device=DEVICE)
test_y = torch.empty((0, action_dim), dtype=torch.float32,device=DEVICE)
for i in range(training_seed):
    train_x = torch.cat((train_x, torch.tensor(data_fullstate[i]["state"].reshape(data_fullstate[i]["state"].shape[0], -1), dtype=torch.float32,device=DEVICE)), dim=0)
    train_y = torch.cat((train_y, torch.tensor(data_fullstate[i]["action_lowpass"].reshape(data_fullstate[i]["action_lowpass"].shape[0], -1), dtype=torch.float32,device=DEVICE)), dim=0)
for j in range(training_seed, num_seeds):
    test_x = torch.cat((test_x, torch.tensor(data_no_joint_pos[j]["state"].reshape(data_no_joint_pos[j]["state"].shape[0], -1), dtype=torch.float32,device=DEVICE)), dim=0)
    test_y = torch.cat((test_y, torch.tensor(data_no_joint_pos[j]["action_lowpass"].reshape(data_no_joint_pos[j]["action_lowpass"].shape[0], -1), dtype=torch.float32,device=DEVICE)), dim=0)

Converge to TensorDataset

In [600]:
state_index = torch.arange(0, 19)
state_dim = len(state_index)

train_dataset = TensorDataset(train_x[:,state_index], train_y)
test_dataset = TensorDataset(test_x[:,state_index], test_y)

print("TRAIN : X , Y shape : ",train_x[:,state_index].shape , train_y.shape)
print("TEST : X , Y shape : ",test_x[:,state_index].shape , test_y.shape)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

TRAIN : X , Y shape :  torch.Size([21000, 19]) torch.Size([21000, 19])
TEST : X , Y shape :  torch.Size([9000, 19]) torch.Size([9000, 19])


### Step 2. Define Encoder

In [601]:
"""
    A simple implementation of Gaussian MLP Encoder
"""
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        z = mean + var*epsilon                          # reparameterization trick
        return z
    
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)         # encoder produces mean and log of variance 
        log_var  = self.FC_var(h_)          # (i.e., parateters of simple tractable normal distribution "q"
        
        z = self.reparameterization(mean, torch.exp(log_var))  # takes exponential function (log var -> var) # reparameterization trick
        # z is sampling from the distribution z = mean + var * epsilon
        return z,mean, log_var

In [602]:
model = Encoder(input_dim=state_dim, hidden_dim=hidden_dim, latent_dim=latent_dim).to(DEVICE)

### Step 3. Define Loss function
- using MSE loss

$$
\mathcal{L}(z,\hat{z}) = \| z^2 - \hat{z}^2 \|^2
$$

In [603]:
MSEloss = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=lr)

def loss_function(x, x_hat, mean, log_var):
    # reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    reproduction_loss = MSEloss(x_hat, x)  # Using MSE loss instead of binary cross entropy
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD


### Step 4. Training network

In [604]:
for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        x = x.view(batch_size, x_dim)
        x = x.to(DEVICE)

        optimizer.zero_grad()

        x_hat, mean, log_var = model(x)
        # loss = loss_function(x, x_hat, mean, log_var)
        loss = MSEloss(x_hat,x)
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    
print("Finish!!")

	Epoch 1 complete! 	Average Loss:  0.0002454508574229429
	Epoch 2 complete! 	Average Loss:  1.8951946173367898e-06
	Epoch 3 complete! 	Average Loss:  6.15748618247598e-07
	Epoch 4 complete! 	Average Loss:  6.477965149080434e-07
	Epoch 5 complete! 	Average Loss:  9.537090975724375e-07
	Epoch 6 complete! 	Average Loss:  9.703886450190903e-07
	Epoch 7 complete! 	Average Loss:  9.046888738047915e-07
	Epoch 8 complete! 	Average Loss:  9.355765779440861e-07
	Epoch 9 complete! 	Average Loss:  9.860106398594904e-07
	Epoch 10 complete! 	Average Loss:  8.287672871115515e-07
	Epoch 11 complete! 	Average Loss:  1.2189468038719566e-06
	Epoch 12 complete! 	Average Loss:  7.598737265565711e-07
	Epoch 13 complete! 	Average Loss:  7.389264263972444e-07
	Epoch 14 complete! 	Average Loss:  9.56544566801504e-07
	Epoch 15 complete! 	Average Loss:  5.708548105845416e-07
	Epoch 16 complete! 	Average Loss:  9.119501496505318e-07
	Epoch 17 complete! 	Average Loss:  6.567881810129342e-07
	Epoch 18 complete! 	Av

In [605]:
model.eval()

with torch.no_grad():
    for batch_idx, (x, _) in enumerate(tqdm.tqdm(test_loader)):
        x = x.view(batch_size, x_dim)
        x = x.to(DEVICE)
        
        x_hat, _, _ = model(x)

100%|██████████| 90/90 [00:00<00:00, 783.85it/s]


In [606]:
test_x[:,state_index].shape


torch.Size([9000, 19])

In [607]:
idx = 4
print(model(test_x[idx , state_index])[0])
print(torch.normal( model(test_x[idx , state_index])[1], torch.exp(model(test_x[idx , state_index])[1])))
print(test_y[idx , state_index])

tensor([ 0.0020,  0.0414, -0.0142, -0.0121, -0.0397, -0.0026,  0.0255,  0.0123,
         0.0194, -0.0125,  0.0339, -0.0033, -0.0336,  0.0052,  0.0018, -0.0195,
         0.0227,  0.0323,  0.0128], device='cuda:0', grad_fn=<AddBackward0>)
tensor([ 0.7372,  1.4372,  0.1268,  0.0081, -0.7416, -0.4339, -1.0284,  0.6189,
        -0.3128,  0.9724,  0.3812, -0.7159,  0.7134,  0.0651, -0.6631, -0.4465,
         1.3070,  0.4355,  1.1686], device='cuda:0', grad_fn=<NormalBackward3>)
tensor([ 0.3439,  0.2428,  0.3439, -0.3416, -0.2206,  0.3439,  0.3117, -0.1444,
        -0.3436, -0.3439, -0.1670,  0.1925, -0.1582, -0.3439, -0.3439, -0.1447,
         0.1695,  0.3404, -0.0277], device='cuda:0')


In [608]:
print( torch.exp(model(test_x[idx , state_index])[2]))

tensor([0.0149, 0.0153, 0.0141, 0.0140, 0.0137, 0.0145, 0.0151, 0.0142, 0.0142,
        0.0148, 0.0144, 0.0146, 0.0143, 0.0143, 0.0145, 0.0141, 0.0150, 0.0143,
        0.0137], device='cuda:0', grad_fn=<ExpBackward0>)


In [609]:
print((model(test_x[idx , state_index])[1]))

tensor([ 0.0162, -0.0014, -0.0135, -0.0236, -0.0172, -0.0156, -0.0043,  0.0094,
         0.0125, -0.0195,  0.0368, -0.0032, -0.0286,  0.0074, -0.0033,  0.0004,
         0.0006,  0.0142,  0.0132], device='cuda:0', grad_fn=<ViewBackward0>)
